## Load Libraries and Data

In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(lubridate))
suppressMessages(library(lattice))
suppressMessages(library(caret))

In [ ]:
#List the Kaggle files.

list.files(path = "../input/ashrae-energy-prediction")

In [ ]:
train <- read_csv("../input/ashrae-energy-prediction/train.csv")
dim(train)

In [ ]:
building_metadata <- read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
dim(building_metadata)

In [ ]:
weather_train <- read_csv("../input/ashrae-energy-prediction/weather_train.csv")
dim(weather_train)

## Join the Data Sets

In [ ]:
train_building <- left_join(train, building_metadata)

In [ ]:
train_building_weather <- left_join(train_building, weather_train)

In [ ]:
glimpse(train_building_weather)
summary(train_building_weather)

## Clean the Data

In [ ]:
# Remove floor_count variable because of excessive NA's.

subset <- train_building_weather %>% select(-floor_count)

In [ ]:
# Count buildings by meter type.

subset %>% group_by(meter, building_id) %>% count() %>% group_by(meter) %>% count()

In [ ]:
# Remove observations with NA's.

subset <- na.omit(subset)

In [ ]:
# Remove outlier observations for meter type 2.

subset <- subset %>% filter(building_id != 1099 | meter != 2)

In [ ]:
# Assess remaining observations.

glimpse(subset)
summary(subset)

In [ ]:
# Verify that remaining observations include some of every meter type in approximately the original proportions.

subset %>% group_by(meter, building_id) %>% count() %>% group_by(meter) %>% count()

In [ ]:
# Remove site_id identifier variable, which is merely a foreign key. (Building_id is similar, but is retained for plotting purposes.)

subset <- subset %>% select(-site_id)

In [ ]:
# Create factors from timestamp variable. Keep timestamp for plotting purposes.

subset <- subset %>%
    mutate(
        week_of_year = week(timestamp),
        day_of_week = wday(timestamp),
        hour_of_day = hour(timestamp))

In [ ]:
# Convert doubles to integers where the values are integer in nature. (Factor type is not used because it interferes with
# the PCA model building, below.)

subset$building_id  <- as.integer(subset$building_id)
subset$meter        <- as.integer(subset$meter)
subset$week_of_year <- as.integer(subset$week_of_year)
subset$day_of_week  <- as.integer(subset$day_of_week)
subset$hour_of_day  <- as.integer(subset$hour_of_day)

In [ ]:
# Glimpse and summarize.

glimpse(subset)
summary(subset)

In [ ]:
# Save a copy of the cleaned data set.

write_csv(subset, "/kaggle/working/subset.csv")

In [ ]:
# This read_csv can remain commented-out unlessed desired upon restarting the kernel to short-circuit the preceding
# data preparation steps.

subset <- read_csv("/kaggle/working/subset.csv")

## Create Subsets by Meter Type for Modeling

In [ ]:
# Create subsets for each meter type.

subset_0 <- subset %>% filter(meter == 0)
subset_1 <- subset %>% filter(meter == 1)
subset_2 <- subset %>% filter(meter == 2)
subset_3 <- subset %>% filter(meter == 3)

Plot meter_reading vs. timestamp for each building.

In [ ]:
ggplot(subset_0, aes(timestamp, meter_reading, group = building_id)) +
  geom_point(size = .25, alpha = .25) +
  facet_wrap(vars(building_id))

In [ ]:
ggplot(subset_1, aes(timestamp, meter_reading, group = building_id)) +
  geom_point(size = .25, alpha = .25) +
  facet_wrap(vars(building_id))

In [ ]:
ggplot(subset_2, aes(timestamp, meter_reading, group = building_id)) +
  geom_point(size = .25, alpha = .25) +
  facet_wrap(vars(building_id))

In [ ]:
ggplot(subset_3, aes(timestamp, meter_reading, group = building_id)) +
  geom_point(size = .25, alpha = .25) +
  facet_wrap(vars(building_id))

## Build Linear Model by Meter Type

In [ ]:
formula  <- as.formula(
 "meter_reading ~
    primary_use +
    square_feet +
    year_built +
    air_temperature +
    cloud_coverage +
    dew_temperature +
    precip_depth_1_hr +
    sea_level_pressure +
    wind_direction +
    wind_speed +
    week_of_year +
    day_of_week +
    hour_of_day")
formula

In [ ]:
lm_0 <- lm(formula, subset_0)
lm_1 <- lm(formula, subset_1)
lm_2 <- lm(formula, subset_2)
lm_3 <- lm(formula, subset_3)

In [ ]:
# Output adjusted R-squared for each linear model.

summary(lm_0)$adj.r.square
summary(lm_1)$adj.r.square
summary(lm_2)$adj.r.square
summary(lm_3)$adj.r.square

Plot residuals from linear model fits by meter type.

In [ ]:
fit_0 <- tibble(lm_0$fitted.values, lm_0$residuals)
glimpse(fit_0)
summary(fit_0)

In [ ]:
ggplot(fit_0, aes(lm_0$fitted.values, lm_0$residuals)) +
  geom_point(size = .1, alpha = .2, color = "blue") +
  geom_hline(yintercept = 0, color = "white", size = 1)

In [ ]:
fit_1 <- tibble(lm_1$fitted.values, lm_1$residuals)
glimpse(fit_1)
summary(fit_1)

In [ ]:
ggplot(fit_1, aes(lm_1$fitted.values, lm_1$residuals)) +
  geom_point(size = .1, alpha = .2, color = "blue") +
  geom_hline(yintercept = 0, color = "white", size = 1)

In [ ]:
fit_2 <- tibble(lm_2$fitted.values, lm_2$residuals)
glimpse(fit_2)
summary(fit_2)

In [ ]:
ggplot(fit_2, aes(lm_2$fitted.values, lm_2$residuals)) +
  geom_point(size = .1, alpha = .2, color = "blue") +
  geom_hline(yintercept = 0, color = "white", size = 1)

In [ ]:
fit_3 <- tibble(lm_3$fitted.values, lm_3$residuals)
glimpse(fit_3)
summary(fit_3)

In [ ]:
ggplot(fit_3, aes(lm_3$fitted.values, lm_3$residuals)) +
  geom_point(size = .1, alpha = .2, color = "blue") +
  geom_hline(yintercept = 0, color = "white", size = 1)

## Principle Component Analysis

In [ ]:
# Split the data set into training and test sets.

set.seed(0)

training.samples <- subset$meter_reading %>% createDataPartition(p = 0.8, list = FALSE)
dim(training.samples)

train.data <- subset[training.samples,]
dim(train.data)

test.data  <- subset[-training.samples,]
dim(test.data)

In [ ]:
# Create training subsets for each meter type, excluding redundant, identifying, and single-valued variables.

train.data.0 <- train.data %>% filter(meter == 0) %>% select(-building_id, -meter, -timestamp)
dim(train.data.0)

train.data.1 <- train.data %>% filter(meter == 1) %>% select(-building_id, -meter, -timestamp)
dim(train.data.1)

train.data.2 <- train.data %>% filter(meter == 2) %>% select(-building_id, -meter, -timestamp)
dim(train.data.2)

train.data.3 <- train.data %>% filter(meter == 3) %>% select(-building_id, -meter, -timestamp)
dim(train.data.3)

In [ ]:
# Create test subsets for each meter type, excluding redundant, identifying, and single-valued variables.

test.data.0 <- test.data %>% filter(meter == 0) %>% select(-building_id, -meter, -timestamp)
dim(test.data.0)

test.data.1 <- test.data %>% filter(meter == 1) %>% select(-building_id, -meter, -timestamp)
dim(test.data.1)

test.data.2 <- test.data %>% filter(meter == 2) %>% select(-building_id, -meter, -timestamp)
dim(test.data.2)

test.data.3 <- test.data %>% filter(meter == 3) %>% select(-building_id, -meter, -timestamp)
dim(test.data.3)

## PCA - Meter Type 0

In [ ]:
# Build models on the training sets.

set.seed(0)

model_0 <- train(
  meter_reading~., data = train.data.0, method = "pcr",
  preProcess = c("center", "scale"),
  trControl = trainControl("cv", number = 5),
  tuneLength = 5)

# Plot model RMSE vs different values of components.
plot(model_0)

# Print the best tuning parameter ncomp that minimizes the cross-validation error, RMSE.
model_0$bestTune

In [ ]:
# Summarize the final model
summary(model_0$finalModel)

In [ ]:
# Make predictions.
predictions <- model_0 %>% predict(test.data.0)

# Model performance metrics
data.frame(
  RMSE = caret::RMSE(predictions, test.data.0$meter_reading),
  Rsquare = caret::R2(predictions, test.data.0$meter_reading))

## PCA - Meter Type 1

In [ ]:
# Build models on the training sets.

set.seed(0)

model_1 <- train(
  meter_reading~., data = train.data.1, method = "pcr",
  preProcess = c("center", "scale"),
  trControl = trainControl("cv", number = 5),
  tuneLength = 5)

# Plot model RMSE vs different values of components.
plot(model_1)

# Print the best tuning parameter ncomp that minimizes the cross-validation error, RMSE.
model_1$bestTune

In [ ]:
# Summarize the final model
summary(model_1$finalModel)

In [ ]:
# Make predictions.
predictions <- model_1 %>% predict(test.data.1)

# Model performance metrics
data.frame(
  RMSE = caret::RMSE(predictions, test.data.1$meter_reading),
  Rsquare = caret::R2(predictions, test.data.1$meter_reading))

## PCA - Meter Type 2

In [ ]:
# Build models on the training sets.

set.seed(0)

model_2 <- train(
  meter_reading~., data = train.data.2, method = "pcr",
  preProcess = c("center", "scale"),
  trControl = trainControl("cv", number = 5),
  tuneLength = 5)

# Plot model RMSE vs different values of components.
plot(model_2)

# Print the best tuning parameter ncomp that minimizes the cross-validation error, RMSE.
model_2$bestTune

In [ ]:
# Summarize the final model
summary(model_2$finalModel)

In [ ]:
# Make predictions.
predictions <- model_2 %>% predict(test.data.2)

# Model performance metrics
data.frame(
  RMSE = caret::RMSE(predictions, test.data.2$meter_reading),
  Rsquare = caret::R2(predictions, test.data.2$meter_reading))

## PCA - Meter Type 3

In [ ]:
# Build models on the training sets.

set.seed(0)

model_3 <- train(
  meter_reading~., data = train.data.3, method = "pcr",
  preProcess = c("center", "scale"),
  trControl = trainControl("cv", number = 5),
  tuneLength = 5)

# Plot model RMSE vs different values of components.
plot(model_3)

# Print the best tuning parameter ncomp that minimizes the cross-validation error, RMSE.
model_3$bestTune

In [ ]:
# Summarize the final model
summary(model_3$finalModel)

In [ ]:
# Make predictions.
predictions <- model_3 %>% predict(test.data.3)

# Model performance metrics
data.frame(
  RMSE = caret::RMSE(predictions, test.data.3$meter_reading),
  Rsquare = caret::R2(predictions, test.data.3$meter_reading))

### Future work
Experiment with prcomp: e.g., prcomp(subset[,c(1:7,10,11)], center = TRUE,scale. = TRUE)